In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import math
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.utils import class_weight
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.models import load_model
import joblib
# import py_vollib.black_scholes.implied_volatility
import multiprocessing
from multiprocessing import Pool
import os
# import py_vollib.black_scholes.implied_volatility as vb
import statistics
%matplotlib inline

Using TensorFlow backend.


In [10]:
# Path settings


path_var = 1 

if (path_var == 1):
    filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'
    modelpath = '/home/docboy/Desktop/Option_Pricing/pickles/'


else:
    filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'
    mdoelpath = '/home/sharan/Desktop/Option_Pricing/pickles/'



nfty = filepath + 'NIFTY50_feature_set_COV_complete.csv'
nfty_ecdf = filepath + 'NIFTY50_feature_set_ECDF_complete.csv'
    
nfty_19 = filepath + 'NIFTY50_2019_feature_set_COV_complete.csv'
nfty_ecdf_19 = filepath + 'NIFTY50_2019_feature_set_ECDF_complete.csv'


bnfty = filepath + 'BANKNIFTY_feature_set_COV_complete.csv'
bnfty_ecdf = filepath + 'BANKNIFTY_feature_set_ECDF_complete.csv'


In [31]:
# Set the automated variables + Plus drop all useless columns approach specific



comb = 0

if (comb == 1):
    u = 'Combined_'
else:
    u = ''


###########################
###############################################

approach = 3                      #------------------>>> # Change value as needed

###############################################
###########################


if (approach == 1):
#     feats = pd.read_csv(nfty_ecdf)
    print ('Approach var set to ECDF')
    
    dropNames = ['date','percentage','strike','spot','opPrice']
    
#     b_feats = pd.read_csv(bnfty_ecdf)
        
    test_feats = pd.read_csv(nfty_ecdf_19)
    
    
else:
#     feats = pd.read_csv(nfty)
    print ('Approach var set to :: '+ str(approach))
    
#     b_feats = pd.read_csv(bnfty)
    test_feats = pd.read_csv(nfty_19)
    
    if (approach == 2):
        dropNames= ['date','percentage','strike','spot','opPrice','change_oi','ts_atr','prevOp','avg_atmError']
        
    else:
        dropNames= ['date', 'percentage','strike','spot','opPrice','ts_atr','change_oi']
        
###################

# X=feats.iloc[:,:-1]
# Y=feats.loc[:,'Target']
# X = X.drop(dropNames, axis = 1)    



# b_X = b_feats.iloc[:,:-1]
# b_Y = b_feats.loc[:,'Target']
# b_X = b_X.drop(dropNames, axis = 1)    



G = test_feats.iloc[:,:-1]
H = test_feats.loc[:,'Target']
G = G.drop(dropNames, axis = 1)  


Approach var set to :: 3


In [32]:
# featNames = list(G)  
# featNames

In [33]:
# Loading the Scalar
scaler = joblib.load(modelpath + u + 'DL_SCALER_NIFTY_0.02_Expt' + str(approach) + '.model')



# Loading the Onehot Encoder
onehot = joblib.load(modelpath + u + 'DL_ONEHOT_NIFTY_0.02_Expt' + str(approach) + '.model')


# Loading the Model
model = joblib.load(modelpath + u + 'DL_NIFTY_0.02_Expt' + str(approach) + '.model')




/home/docboy/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/docboy/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.22 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [34]:
def c_int(lst):
    df = pd.DataFrame(lst)
    df = df.sort_values(by = 0).reset_index(drop=True)
    cp1 = df[np.logical_and(df[0] >= -2,df[0] <= 2)]
    return (len(cp1)/len(df))*100

In [35]:

featNames = list(G)  
G = scaler.transform(G)
G = pd.DataFrame(G, columns = featNames)

H = H.values
H = H.reshape(len(H), 1)
H = onehot.fit_transform(H)


X_test = G


Y_test = H


Y_pred=model.predict(X_test)
Y_pred=np.argmax(Y_pred,axis=1)

OGY_test=onehot.inverse_transform(Y_test)
        # OGY_test=sum(OGY_test, [])
OGY_test=OGY_test.ravel()
# print(OGY_test)


width = 0.1

num_bins = 50

bin_width = width/num_bins

        # Normalized Error Metric (NEM)

dist_sum = 0
for i in range(len(Y_pred)):
    dist = abs(Y_pred[i] - OGY_test[i])
    dist_sum = dist_sum + dist

print("NEM:{}".format((dist_sum*bin_width)/len(Y_test)))

CP=[]
for i in range(len(Y_pred)):
    dist = OGY_test[i] - Y_pred[i]
    CP.append(dist)
CP=np.array(CP,dtype=float)

print("Confidence Interval:{}".format(c_int(CP)))


NEM:0.0051099517412286416
Confidence Interval:65.91887309247424


In [30]:
G

,close_mean,close_sd,open_mean,open_sd,high_mean,high_sd,low_mean,low_sd,close_lr_open_lr_cov,close_lr_high_lr_cov,close_lr_low_lr_cov,open_lr_high_lr_cov,open_lr_low_lr_cov,high_lr_low_lr_cov,avg_atmError,prevOp,ttm,atmError,yield03
0,0.989172,-0.620275,1.197223,-0.366548,1.020293,-0.345521,1.152269,-0.516661,-0.226898,-0.349883,-0.552269,-0.432838,-0.196941,-0.286625,0.402973,1.205195,0.15,1.105441,-0.244898
1,0.989172,-0.620275,1.197223,-0.366548,1.020293,-0.345521,1.152269,-0.516661,-0.226898,-0.349883,-0.552269,-0.432838,-0.196941,-0.286625,-0.333819,-0.201500,-0.55,0.245429,-0.244898
2,0.989172,-0.620275,1.197223,-0.366548,1.020293,-0.345521,1.152269,-0.516661,-0.226898,-0.349883,-0.552269,-0.432838,-0.196941,-0.286625,-0.833564,-0.524497,-0.55,-0.337893,-0.244898
3,0.989172,-0.620275,1.197223,-0.366548,1.020293,-0.345521,1.152269,-0.516661,-0.226898,-0.349883,-0.552269,-0.432838,-0.196941,-0.286625,-1.027632,-0.588676,-0.55,-0.564416,-0.244898
4,0.989172,-0.620275,1.197223,-0.366548,1.020293,-0.345521,1.152269,-0.516661,-0.226898,-0.349883,-0.552269,-0.432838,-0.196941,-0.286625,-0.127864,0.060749,-0.55,0.485828,-0.244898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7662,-2.589680,11.398021,-0.125975,9.129135,-2.245102,8.099684,0.390247,8.656402,7.858785,10.716977,8.961172,6.895555,9.507244,8.951136,0.014648,2.066852,-0.60,0.810780,-2.387755
7663,-2.589680,11.398021,-0.125975,9.129135,-2.245102,8.099684,0.390247,8.656402,7.858785,10.716977,8.961172,6.895555,9.507244,8.951136,-0.259628,1.537690,-0.60,0.488832,-2.387755
7664,-2.589680,11.398021,-0.125975,9.129135,-2.245102,8.099684,0.390247,8.656402,7.858785,10.716977,8.961172,6.895555,9.507244,8.951136,-0.527741,1.225811,-0.60,0.174118,-2.387755
7665,-2.589680,11.398021,-0.125975,9.129135,-2.245102,8.099684,0.390247,8.656402,7.858785,10.716977,8.961172,6.895555,9.507244,8.951136,-0.789896,0.888248,-0.60,-0.133602,-2.387755
